In [1]:
%%bash
# Clone the GitHub repo
git clone https://github.com/isacancabuk/toyota-model-labelling.git /content/toyota-model-labelling
cd /content/toyota-model-labelling

# Install Python packages from your cleaned requirements.txt
pip install -q -r requirements.txt

fatal: destination path '/content/toyota-model-labelling' already exists and is not an empty directory.


In [2]:
# Cell 2: one-time imports and dataset inspection
import os
from pathlib import Path

REPO = Path('/content/toyota-model-labelling')
DATA_4R = REPO / 'data' / '4runner'

if not DATA_4R.exists():
    raise FileNotFoundError(f"Folder not found: {DATA_4R}")

files = list(DATA_4R.glob('*.jpg'))
print("Total 4Runner images:", len(files))
print("First 5 filenames:", [f.name for f in files[:5]])

Total 4Runner images: 946
First 5 filenames: ['4runner-1096240004.jpg', '4runner-582877487.jpg', '4runner-1138715470.jpg', '4runner-1691680277.jpg', '4runner-1668485905.jpg']


In [4]:
# Cell 3: split the 4Runner images into train/val (80/20)
import random, shutil

SRC = REPO / 'data' / '4runner'
TRAIN = REPO / 'data' / '4runner_split' / 'train'
VAL   = REPO / 'data' / '4runner_split' / 'val'

for d in (TRAIN, VAL):
    d.mkdir(parents=True, exist_ok=True)

imgs = list(SRC.glob('*.jpg'))
random.shuffle(imgs)
cut = int(0.8 * len(imgs))

for i, img in enumerate(imgs):
    tgt = TRAIN if i < cut else VAL
    shutil.copy(img, tgt / img.name)

print(f"Train count: {len(list(TRAIN.glob('*')))}")
print(f" Val count: {len(list(VAL.glob('*')))}")

Train count: 929
 Val count: 353


In [3]:
%%bash
# Cell 4: get YOLOv5 code & install its requirements
git clone https://github.com/ultralytics/yolov5.git /content/yolov5
cd /content/yolov5
pip install -q -r requirements.txt

fatal: destination path '/content/yolov5' already exists and is not an empty directory.


In [4]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Cell 5: baseline recall for 'car' on your Val set (fixed conf handling)
import torch
from pathlib import Path

# 1) Load the COCO-pretrained YOLOv5s
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# 2) Set minimum confidence once
model.conf = 0.25  # equivalent to passing conf_thres=0.25 each call

CAR_ID = 2
val_images = list((REPO / 'data' / '4runner_split' / 'val').glob('*.jpg'))
total = len(val_images)
detected = 0

# 3) Loop and count
for img in val_images:
    results = model(str(img))         # now uses model.conf threshold
    df = results.pandas().xyxy[0]     # DataFrame of all detections
    if (df['class'] == CAR_ID).any():
        detected += 1

# 4) Report recall
print(f"Detected in {detected}/{total} images → Recall: {detected/total:.2%}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-15 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Detected in 105/190 images → Recall: 55.26%


In [5]:
%%bash
# Search every folder under /content for 4runner.yaml
echo "Searching for 4runner.yaml..."
find /content -type f -name 4runner.yaml

Searching for 4runner.yaml...


In [6]:
!ls -l /content/toyota-model-labelling/data/4runner.yaml

ls: cannot access '/content/toyota-model-labelling/data/4runner.yaml': No such file or directory


In [7]:
%%bash
cat > /content/toyota-model-labelling/data/4runner_abs.yaml << 'EOF'
train: /content/toyota-model-labelling/data/4runner_split/train
val:   /content/toyota-model-labelling/data/4runner_split/val

nc: 1
names: ['4runner']
EOF

In [15]:
%%bash
export WANDB_MODE=disabled
cd /content/yolov5

python train.py \
  --img 640 \
  --batch 8 \
  --epochs 50 \
  --data /content/toyota-model-labelling/data/4runner_abs.yaml \
  --weights yolov5s.pt \
  --name 4runner_finetune_abs \
  --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-05-15 12:41:30.744382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747312890.764979   15879 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747312890.771405   15879 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/toyota-model-labelling/data/4runner_abs.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_pop

In [21]:
df = results.pandas().xyxy[0]
print(df['class'].unique())  # Tahmin edilen sınıf ID’lerini yazdırın

[]


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  